In [101]:
import pandas as pd

melb_data = pd.read_csv("data/melb_data_ps.csv")
melb_df = melb_data.copy()

# Создание и преобразование столбцов с помощью функций

Мы можем написать некоторую функцию, которая принимает на вход один элемент столбца, каким-то образом его обрабатывает и возвращает результат, после чего применить эту функцию к каждому элементу в столбце с помощью специального метода <code>apply()</code>. В результате применения этой функции будет возвращён объект Series, элементы которого будут представлять результат работы этой функции.


In [102]:
print(melb_df["Address"].nunique())

13378


Обычно подобные признаки удаляют, однако можно поступить умнее: давайте извлечём из признака адреса характеристику подтипа улицы (улица, шоссе, авеню, бульвар). Для этого сначала внимательнее посмотрим на структуру адреса, выберем несколько строк столбца Address:


In [103]:
print(melb_df["Address"].loc[177])
print(melb_df["Address"].loc[1812])
print(melb_df["Address"].loc[9001])

2/119 Railway St N
9/400 Dandenong Rd
172 Danks St


Итак, адрес строится следующим образом: сначала указывается номер дома и корпус, после указывается название улицы, а в конце — подтип улицы, но в некоторых случаях к подтипу добавляется географическая отметка (N — север, S — юг и т. д.), она нам не нужна . Для того чтобы выделить подтип улицы, на которой находится объект, можно использовать следующую функцию:


In [104]:
# На вход данной функции поступает строка с адресом.
def get_street_type(address):
    # Создаём список географических пометок exclude_list.
    exclude_list = ["N", "S", "W", "E"]
    full_names = ["Avenue", "Boulevard", "Parade"]
    short_names = ["Av", "Bvd", "Pde"]

    # Метод split() разбивает строку на слова по пробелу.
    # В результате получаем список слов в строке и заносим его в переменную address_list.
    address_list = address.split(" ")
    # Обрезаем список, оставляя в нём только последний элемент,
    # потенциальный подтип улицы, и заносим в переменную street_type.
    street_type = address_list[-1]
    # Делаем проверку на то, что полученный подтип является географической пометкой.
    # Для этого проверяем его на наличие в списке exclude_list.
    if street_type in exclude_list:
        # Если переменная street_type является географической пометкой,
        # переопределяем её на второй элемент с конца списка address_list.
        street_type = address_list[-2]

    street_type = (
        short_names[full_names.index(street_type)]
        if street_type in full_names
        else street_type
    )
    # Возвращаем переменную street_type, в которой хранится подтип улицы.
    return street_type

Теперь применим эту функцию к столбцу c адресом. Для этого передадим функцию get_street_type в аргумент метода столбца apply(). В результате получим объект Series, который положим в переменную street_types:


In [105]:
street_types = melb_df["Address"].apply(get_street_type)
display(street_types)

0        St
1        St
2        St
3        La
4        St
         ..
13575    Cr
13576    Dr
13577    St
13578    St
13579    St
Name: Address, Length: 13580, dtype: object

Итак, мы смогли выделить подтип улицы. Посмотрим, сколько уникальных значений у нас получилось:


In [106]:
print(street_types.nunique())

53


У нас есть 56 уникальных значений. Однако наш результат можно улучшить. Давайте для начала посмотрим на частоту каждого подтипа улицы с помощью метода value_counts:


In [107]:
display(street_types.value_counts())

Address
St           8012
Rd           2825
Ct            612
Dr            447
Av            361
Gr            311
Pde           226
Pl            169
Cr            152
Cl            100
La             67
Bvd            66
Tce            47
Wy             40
Cct            25
Hwy            24
Sq             11
Crescent        9
Cir             7
Strand          7
Esplanade       6
Grove           5
Mews            4
Grn             4
Fairway         4
Gdns            4
Righi           3
Crossway        3
Esp             2
Victoria        2
Ridge           2
Crofts          2
Grand           1
Summit          1
Hts             1
Athol           1
Highway         1
Outlook         1
Woodland        1
Ave             1
Gra             1
Terrace         1
Eyrie           1
Dell            1
East            1
Loop            1
Nook            1
Glade           1
Qy              1
Cove            1
Res             1
Grange          1
Corso           1
Name: count, dtype: int64

В таком случае давайте применим очень распространённый метод уменьшения количества уникальных категорий — выделим n подтипов, которые встречаются чаще всего, а остальные обозначим как 'other' (другие).

Для этого к результату метода value_counts применим метод nlargest(), который возвращает n наибольших значений из Series. Зададим n=10, т. е. мы хотим отобрать десять наиболее популярных подтипов. Извлечём их названия с помощью атрибута index, а результат занесём в переменную popular_stypes:


In [108]:
popular_stypes = street_types.value_counts().nlargest(10).index
print(popular_stypes)

Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object', name='Address')


Теперь, когда у нас есть список наиболее популярных подтипов улиц, введём lambda-функцию, которая будет проверять, есть ли строка x в этом перечне, и, если это так, lambda-функция будет возвращать x, в противном случае она будет возвращать строку 'other'. Наконец, применим такую функцию к Series street_types, полученной ранее, а результат определим в новый столбец таблицы StreetType:


In [109]:
melb_df["StreetType"] = street_types.apply(
    lambda x: x if x in popular_stypes else "other"
)
display(melb_df["StreetType"])

0           St
1           St
2           St
3        other
4           St
         ...  
13575       Cr
13576       Dr
13577       St
13578       St
13579       St
Name: StreetType, Length: 13580, dtype: object

In [110]:
print(melb_df["StreetType"].nunique())

11


Теперь, у нас нет потребности хранить признак Address, так как, если конкретное местоположение объекта всё же и влияет на его стоимость, то оно определяется столбцами Longitude и Lattitude. Удалим его из нашей таблицы:


In [111]:
melb_df = melb_df.drop("Address", axis=1)

Резюмируя, поделимся общими рекомендациями по уменьшению числа уникальных значений в признаке, который описывается категориями:

1. Определите (хотя бы на глаз) соотношение числа уникальных категорий интересующего вас признака к общему числу объектов в таблице. Если это соотношение превышает значение 30 %, то это уже повод задуматься над уменьшением числа категорий и перейти к шагу 2.

2. Если ваш признак уникален для каждого объекта, например адрес, имя или название, то такой признак, скорее всего, не имеет статистической значимости. От таких признаков чаще всего избавляются. Однако можно попробовать выделить из этого признака какие-то общие черты, например, как мы это сделали с подтипами улиц. Такой же трюк можно произвести, например, с названиями компаний, в которых может быть скрыт признак типа организации (из строки «ООО Три Слепые Мыши» можно извлечь ООО — общество с ограниченной ответственностью).

3. Если даже после преобразования число уникальных категорий всё ещё велико, можно попробовать с помощью метода value_counts() оценить, есть ли в данных категории, которые употребляются гораздо реже, чем остальные. Если такие категории присутствуют, переходите к шагу 4.

4. Можно подобрать число популярных категорий таким образом, чтобы эти категории покрывали большую часть ваших данных.

5. Наконец, можно совершить преобразование, обозначив категории, не попавшие в число популярных, как «другие».


In [112]:
melb_data = pd.read_csv("./data/melb_data_ps.csv")
melb_df = melb_data.copy()

melb_df["Date"] = pd.to_datetime(melb_df["Date"], dayfirst=True)
melb_df["WeekdaySale"] = melb_df["Date"].dt.dayofweek


def get_weekend(weekday):
    return 1 if weekday in [5, 6] else 0


melb_df["Weekend"] = melb_df["WeekdaySale"].apply(get_weekend)

#  вычислите среднюю цену объекта недвижимости, проданного в выходные дни. Результат округлите до целых.


mean_weekend_price = round(melb_df[melb_df["Weekend"] == 1]["Price"].mean())
display(mean_weekend_price)

# Преобразуйте столбец SellerG с наименованиями риелторских компаний в таблице melb_df следующим образом: оставьте в столбце только 49 самых популярных компаний, а остальные обозначьте как 'other'.
# Найдите, во сколько раз минимальная цена объектов недвижимости, проданных компанией 'Nelson', больше минимальной цены объектов, проданных компаниями, обозначенными как 'other'. Ответ округлите до десятых.


sellers = melb_df["SellerG"].value_counts().nlargest(49)


def get_sellers(seller):
    if seller in sellers:
        return seller
    else:
        return "other"


melb_df["SellerG"] = melb_df["SellerG"].apply(get_sellers)
nelson_sales = (
    melb_df[melb_df["SellerG"] == "Nelson"]["Price"].min()
    / melb_df[melb_df["SellerG"] == "other"]["Price"].min()
)
nelson_sales

1081199

1.297709923664122